In [1]:
from tensorflow import keras
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.0)

In [2]:
def Inception(x,nb_filter):  
    branch1x1 = keras.layers.Conv2D(nb_filter,(1,1),padding='same',strides=(1,1),activation='relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(x)  

    branch3x3 = keras.layers.Conv2D(nb_filter,(1,1),padding='same',strides=(1,1),activation='relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(x) 
    branch3x31 = keras.layers.Conv2D(nb_filter,(3,1),padding='same',strides=(1,1),activation='relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(branch3x3)  
    branch3x32 = keras.layers.Conv2D(nb_filter,(1,3),padding='same',strides=(1,1),activation='relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(branch3x3) 
    out1=keras.layers.Add()([branch3x31, branch3x32])
    
    branch5x5 = keras.layers.Conv2D(nb_filter,(1,1),padding='same',strides=(1,1),activation='relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(x) 
    branch5x5_1 = keras.layers.Conv2D(nb_filter,(3,1),padding='same',strides=(1,1),activation='relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(branch5x5) 
    branch5x5_2 = keras.layers.Conv2D(nb_filter,(1,3),padding='same',strides=(1,1),activation='relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(branch5x5) 
    out2=keras.layers.Add()([branch5x5_1, branch5x5_2])

    branch5x51 = keras.layers.Conv2D(nb_filter,(3,1),padding='same',strides=(1,1),activation='relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(out2) 
    branch5x52 = keras.layers.Conv2D(nb_filter,(1,3),padding='same',strides=(1,1),activation='relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(out2) 
    out3=keras.layers.Add()([branch5x51, branch5x52])

    branchpool = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same')(x)  
    branchpool = keras.layers.Conv2D(nb_filter,(1,1),padding='same',strides=(1,1),activation='relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(branchpool) 
  
    x = keras.layers.concatenate([branch1x1,out1,out3,branchpool],axis=3)    
    return x  


In [3]:
import tensorflow as tf
base_model=tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

In [4]:
x = base_model.layers[-13].output
x = Inception(x,256)
x = Inception(x,256)
x = Inception(x,256)
x = keras.layers.GlobalAveragePooling2D()(x)
predictions = keras.layers.Dense(38, activation='softmax')(x)
model = keras.models.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='sgd', loss='categorical_crossentropy')
model.summary()

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [6]:
learning_rate = 0.0001  
adam_optimizer=keras.optimizers.Adam(learning_rate = learning_rate)
model.compile(loss = 'categorical_crossentropy', optimizer = adam_optimizer, metrics = ['accuracy'])
